In [ ]:
#| default_exp losses

# Losses

> NumPy loss functions for model evaluation.
> 

> The most important train signal is the forecast error, which is the difference between the observed value $y_{\tau}$ and the prediction $\hat{y}_{\tau}$, at time $y_{\tau}$:
> 
> $$ e_{\tau} = y_{\tau}-\hat{y}_{\tau} \qquad \qquad \tau \in \{t+1,\dots,t+H \} $$
> 
> The train loss summarizes the forecast errors in different evaluation metrics.

In [ ]:
#| export
from typing import Optional, Union

import numpy as np

In [ ]:
#| hide
import warnings

from nbdev import show_doc

In [ ]:
#| hide
warnings.filterwarnings('ignore', message='Unknown section References')

In [ ]:
#| exporti
def _divide_no_nan(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Auxiliary funtion to handle divide by 0"""
    out_dtype = np.result_type(np.float32, a.dtype, b.dtype)
    return np.divide(a, b, out=np.zeros(a.shape, dtype=out_dtype), where=b!=0)

In [ ]:
#| exporti
def _metric_protections(
    y: np.ndarray, y_hat: np.ndarray, weights: Optional[np.ndarray] = None
) -> None:
    if weights is None:
        return
    if np.sum(weights) <= 0:
        raise ValueError('Sum of weights must be positive')
    if y.shape != y_hat.shape:
        raise ValueError(f'Wrong y_hat dimension. y_hat shape={y_hat.shape}, y shape={y.shape}')
    if weights.shape != y.shape:
        raise ValueError(f'Wrong weight dimension. weights shape={weights.shape}, y shape={y.shape}')

# <span style="color:DarkOrange">1. Scale-dependent Errors </span>

## Mean Absolute Error (MAE)
> $$ \mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} |y_{\tau} - \hat{y}_{\tau}| $$

![](imgs/losses/mae_loss.png)

In [ ]:
#| export
def mae(
    y: np.ndarray,
    y_hat: np.ndarray,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Mean Absolute Error (MAE)

    MAE measures the relative prediction
    accuracy of a forecasting method by calculating the
    deviation of the prediction and the true
    value at a given time and averages these devations
    over the length of the series.

    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from last to first.

    Returns
    -------
    numpy array or double
        MAE along the specified axis.
    """
    _metric_protections(y, y_hat, weights)
    
    delta_y = np.abs(y - y_hat)
    if weights is not None:
        mae = np.average(delta_y[~np.isnan(delta_y)], 
                         weights=weights[~np.isnan(delta_y)],
                         axis=axis)
    else:
        mae = np.nanmean(delta_y, axis=axis)
    return mae

In [ ]:
show_doc(mae)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L36){target="_blank" style="float:right; font-size:smaller"}

### mae

>      mae (y:numpy.ndarray, y_hat:numpy.ndarray,
>           weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Mean Absolute Error (MAE)

MAE measures the relative prediction
accuracy of a forecasting method by calculating the
deviation of the prediction and the true
value at a given time and averages these devations
over the length of the series.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from last to first. |
| **Returns** | **Union** |  | **MAE along the specified axis.** |

## Mean Squared Error
> $$ \mathrm{MSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2} $$

![](imgs/losses/mse_loss.png)

In [ ]:
#| export
def mse(
    y: np.ndarray,
    y_hat: np.ndarray, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Mean Squared Error (MSE)
    
    MSE measures the relative prediction
    accuracy of a forecasting method by calculating the 
    squared deviation of the prediction and the true
    value at a given time, and averages these devations
    over the length of the series.
    
    Parameters
    ----------
    y : numpy array
        Actual test values.
    y_hat : numpy array
        Predicted values.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the 
        elements of the input array. If axis is negative it counts 
        from the last to the first axis.

    Returns
    -------
    numpy array or double
        MSE along the specified axis.
    """
    _metric_protections(y, y_hat, weights)

    delta_y = np.square(y - y_hat)
    if weights is not None:
        mse = np.average(delta_y[~np.isnan(delta_y)], 
                         weights=weights[~np.isnan(delta_y)], 
                         axis=axis)
    else:
        mse = np.nanmean(delta_y, axis=axis)
        
    return mse

In [ ]:
show_doc(mse)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L80){target="_blank" style="float:right; font-size:smaller"}

### mse

>      mse (y:numpy.ndarray, y_hat:numpy.ndarray,
>           weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Mean Squared Error (MSE)

MSE measures the relative prediction
accuracy of a forecasting method by calculating the 
squared deviation of the prediction and the true
value at a given time, and averages these devations
over the length of the series.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Actual test values. |
| y_hat | ndarray |  | Predicted values. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the <br>elements of the input array. If axis is negative it counts <br>from the last to the first axis. |
| **Returns** | **Union** |  | **MSE along the specified axis.** |

## Root Mean Squared Error
> $$ \mathrm{RMSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \sqrt{\frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2}} $$

![](imgs/losses/rmse_loss.png)

In [ ]:
#| export
def rmse(
    y: np.ndarray,
    y_hat: np.ndarray,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Root Mean Squared Error (RMSE)
    
    RMSE measures the relative prediction
    accuracy of a forecasting method by calculating the squared deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale. 
    RMSE has a direct connection to the L2 norm.

    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        RMSE along the specified axis.
    """

    return np.sqrt(mse(y, y_hat, weights, axis))

In [ ]:
show_doc(rmse)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L126){target="_blank" style="float:right; font-size:smaller"}

### rmse

>      rmse (y:numpy.ndarray, y_hat:numpy.ndarray,
>            weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Root Mean Squared Error (RMSE)

RMSE measures the relative prediction
accuracy of a forecasting method by calculating the squared deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
Finally the RMSE will be in the same scale
as the original time series so its comparison with other
series is possible only if they share a common scale. 
RMSE has a direct connection to the L2 norm.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **RMSE along the specified axis.** |

# <span style="color:DarkOrange">2. Percentage Errors </span>

## Mean Absolute Percentage Error
> $$ \mathrm{MAPE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|} $$

![](imgs/losses/mape_loss.png)

In [ ]:
#| export
def mape(
    y: np.ndarray,
    y_hat: np.ndarray, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Mean Absolute Percentage Error (MAPE)
    
    MAPE measures the relative prediction
    accuracy of a forecasting method by calculating the percentual deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    The closer to zero an observed value is, the higher penalty MAPE loss
    assigns to the corresponding error.
    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        MAPE along the specified axis.
    """
    _metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y)
    mape = _divide_no_nan(delta_y, scale)
    mape = np.average(mape, weights=weights, axis=axis)
    mape = 100 * mape
    
    return mape

In [ ]:
show_doc(mape)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L165){target="_blank" style="float:right; font-size:smaller"}

### mape

>      mape (y:numpy.ndarray, y_hat:numpy.ndarray,
>            weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Mean Absolute Percentage Error (MAPE)

MAPE measures the relative prediction
accuracy of a forecasting method by calculating the percentual deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
The closer to zero an observed value is, the higher penalty MAPE loss
assigns to the corresponding error.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **MAPE along the specified axis.** |

## Symmetric Mean Absolute Percentage Error
> $$ \mathrm{SMAPE}_{2}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|+|\hat{y}_{\tau}|} $$

In [ ]:
#| export
def smape(
    y: np.ndarray,
    y_hat: np.ndarray,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Symmetric Mean Absolute Percentage Error (SMAPE)

    SMAPE measures the relative prediction
    accuracy of a forecasting method by calculating the relative deviation
    of the prediction and the observed value scaled by the sum of the
    absolute values for the prediction and observed value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 200% which is desireble compared to normal MAPE that
    may be undetermined when the target is zero.

    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : None or int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        SMAPE along the specified axis.
    """
    _metric_protections(y, y_hat, weights)
        
    delta_y = np.abs(y - y_hat)
    scale = np.abs(y) + np.abs(y_hat)
    smape = _divide_no_nan(delta_y, scale)
    smape = 200 * np.average(smape, weights=weights, axis=axis)
    
    if isinstance(smape, float):
        assert smape <= 200, 'SMAPE should be lower than 200'
    else:
        assert all(smape <= 200), 'SMAPE should be lower than 200'
    
    return smape

In [ ]:
show_doc(smape)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L209){target="_blank" style="float:right; font-size:smaller"}

### smape

>      smape (y:numpy.ndarray, y_hat:numpy.ndarray,
>             weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Symmetric Mean Absolute Percentage Error (SMAPE)

SMAPE measures the relative prediction
accuracy of a forecasting method by calculating the relative deviation
of the prediction and the observed value scaled by the sum of the
absolute values for the prediction and observed value at a
given time, then averages these devations over the length
of the series. This allows the SMAPE to have bounds between
0% and 200% which is desireble compared to normal MAPE that
may be undetermined when the target is zero.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **SMAPE along the specified axis.** |

# <span style="color:DarkOrange">3. Scale-independent Errors </span>

## Mean Absolute Scaled Error
> $$ \mathrm{MASE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau}) = 
        \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})} $$

![](imgs/losses/mase_loss.png)

In [ ]:
#| export
def mase(
    y: np.ndarray,
    y_hat: np.ndarray,
    y_train: np.ndarray,
    seasonality: int,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Mean Absolute Scaled Error (MASE)
    
    MASE measures the relative prediction
    accuracy of a forecasting method by comparinng the mean absolute errors
    of the prediction and the observed value against the mean
    absolute errors of the seasonal naive model.
    The MASE partially composed the Overall Weighted Average (OWA), 
    used in the M4 Competition.

    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.   
    y_train : numpy array
        Actual insample Seasonal Naive predictions.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        MASE along the specified axis.

    References
    ----------
    [1] https://robjhyndman.com/papers/mase.pdf
    """    
    delta_y = np.abs(y - y_hat)
    delta_y = np.average(delta_y, weights=weights, axis=axis)
    
    scale = np.abs(y_train[:-seasonality] - y_train[seasonality:])
    scale = np.average(scale, axis=axis)
    
    mase = delta_y / scale
    
    return mase

In [ ]:
show_doc(mase)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L260){target="_blank" style="float:right; font-size:smaller"}

### mase

>      mase (y:numpy.ndarray, y_hat:numpy.ndarray, y_train:numpy.ndarray,
>            seasonality:int, weights:Optional[numpy.ndarray]=None,
>            axis:Optional[int]=None)

Mean Absolute Scaled Error (MASE)

MASE measures the relative prediction
accuracy of a forecasting method by comparinng the mean absolute errors
of the prediction and the observed value against the mean
absolute errors of the seasonal naive model.
The MASE partially composed the Overall Weighted Average (OWA), 
used in the M4 Competition.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.    |
| y_train | ndarray |  | Actual insample Seasonal Naive predictions. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **MASE along the specified axis.** |

## Relative Mean Absolute Error
> $$ \mathrm{RMAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau})} $$

![](imgs/losses/rmae_loss.png)

In [ ]:
#| export
def rmae(
    y: np.ndarray,
    y_hat1: np.ndarray,
    y_hat2: np.ndarray,
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Relative Mean Absolute Error (RMAE)
    
    Calculates the RAME between two sets of forecasts (from two different forecasting methods).
    A number smaller than one implies that the forecast in the 
    numerator is better than the forecast in the denominator.
    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat1 : numpy array
        Predicted values of first model.
    y_hat2 : numpy array
        Predicted values of baseline model.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a.
        The default, axis=None, will average over all of the elements of
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        RMAE along the specified axis.
    """
    numerator = mae(y=y, y_hat=y_hat1, weights=weights, axis=axis)
    denominator = mae(y=y, y_hat=y_hat2, weights=weights, axis=axis)
    rmae = numerator / denominator
    
    return rmae

In [ ]:
show_doc(rmae)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L315){target="_blank" style="float:right; font-size:smaller"}

### rmae

>      rmae (y:numpy.ndarray, y_hat1:numpy.ndarray, y_hat2:numpy.ndarray,
>            weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Relative Mean Absolute Error (RMAE)

Calculates the RAME between two sets of forecasts (from two different forecasting methods).
A number smaller than one implies that the forecast in the 
numerator is better than the forecast in the denominator.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat1 | ndarray |  | Predicted values of first model. |
| y_hat2 | ndarray |  | Predicted values of baseline model. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a.<br>The default, axis=None, will average over all of the elements of<br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **RMAE along the specified axis.** |

# <span style="color:DarkOrange">4. Probabilistic Errors </span>

## Quantile Loss
> $$ \mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q)}_{\tau}) = 
        \frac{1}{H} \sum^{t+H}_{\tau=t+1} 
        \Big( (1-q)\,( \hat{y}^{(q)}_{\tau} - y_{\tau} )_{+} 
        + q\,( y_{\tau} - \hat{y}^{(q)}_{\tau} )_{+} \Big) $$

![](imgs/losses/q_loss.png)

In [ ]:
#| export
def quantile_loss(
    y: np.ndarray,
    y_hat: np.ndarray,
    q: float = 0.5, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Quantile Loss (QL)
    
    QL measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.    
    A common value for q is 0.5 for the deviation from the median.       
            
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    q : float (default=0.5)
        Quantile for the predictions' comparison.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        QL along the specified axis.
    """
    _metric_protections(y, y_hat, weights)

    delta_y = y - y_hat
    loss = np.maximum(q * delta_y, (q - 1) * delta_y)

    if weights is not None:
        quantile_loss = np.average(loss[~np.isnan(loss)], 
                             weights=weights[~np.isnan(loss)],
                             axis=axis)
    else:
        quantile_loss = np.nanmean(loss, axis=axis)
        
    return quantile_loss

In [ ]:
show_doc(quantile_loss)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L355){target="_blank" style="float:right; font-size:smaller"}

### quantile_loss

>      quantile_loss (y:numpy.ndarray, y_hat:numpy.ndarray, q:float=0.5,
>                     weights:Optional[numpy.ndarray]=None,
>                     axis:Optional[int]=None)

Quantile Loss (QL)

QL measures the deviation of a quantile forecast.
By weighting the absolute deviation in a non symmetric way, the
loss pays more attention to under or over estimation.    
A common value for q is 0.5 for the deviation from the median.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| q | float | 0.5 | Quantile for the predictions' comparison. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **QL along the specified axis.** |

## Multi-Quantile Loss
> $$ \mathrm{MQL}(\mathbf{y}_{\tau},
                    [\mathbf{\hat{y}}^{(q_{1})}_{\tau}, ... ,\hat{y}^{(q_{n})}_{\tau}]) = 
       \frac{1}{n} \sum_{q_{i}} \mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q_{i})}_{\tau}) $$

![](imgs/losses/mq_loss.png)

In [ ]:
#| export
def mqloss(
    y: np.ndarray,
    y_hat: np.ndarray, 
    quantiles: np.ndarray, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """ Multi-Quantile loss (MQL)
    
    MQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute 
    difference between predicted quantiles and observed values.

    The limit behavior of MQL allows to measure the accuracy 
    of a full predictive distribution $\mathbf{\hat{F}}_{\\tau}$ with 
    the continuous ranked probability score (CRPS). This can be achieved 
    through a numerical integration technique, that discretizes the quantiles 
    and treats the CRPS integral with a left Riemann approximation, averaging over 
    uniformly distanced quantiles.    

    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    quantiles : numpy array
        Quantiles to compare against.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double
        MQL along the specified axis.

    References
    ----------
    [1] https://www.jstor.org/stable/2629907
    """ 
    if weights is None: weights = np.ones(y.shape)
        
    _metric_protections(y, y_hat, weights)
    n_q = len(quantiles)
    
    y_rep  = np.expand_dims(y, axis=-1)
    error  = y_rep - y_hat
    mqloss = np.maximum(quantiles * error, (quantiles - 1) * error)
    
    # Match y/weights dimensions and compute weighted average
    weights = np.repeat(np.expand_dims(weights, axis=-1), repeats=n_q, axis=-1)
    mqloss  = np.average(mqloss, weights=weights, axis=axis)

    return mqloss

In [ ]:
show_doc(mqloss)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L404){target="_blank" style="float:right; font-size:smaller"}

### mqloss

>      mqloss (y:numpy.ndarray, y_hat:numpy.ndarray, quantiles:numpy.ndarray,
>              weights:Optional[numpy.ndarray]=None, axis:Optional[int]=None)

Multi-Quantile loss (MQL)

MQL calculates the average multi-quantile Loss for
a given set of quantiles, based on the absolute 
difference between predicted quantiles and observed values.

The limit behavior of MQL allows to measure the accuracy 
of a full predictive distribution $\mathbf{\hat{F}}_{\tau}$ with 
the continuous ranked probability score (CRPS). This can be achieved 
through a numerical integration technique, that discretizes the quantiles 
and treats the CRPS integral with a left Riemann approximation, averaging over 
uniformly distanced quantiles.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| quantiles | ndarray |  | Quantiles to compare against. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **MQL along the specified axis.** |

## Coverage

In [ ]:
#| export
def coverage(
    y: np.ndarray,
    y_hat_lo: np.ndarray,
    y_hat_hi: np.ndarray, 
) -> Union[float, np.ndarray]:
    """
    Coverage of y with y_hat_lo and y_hat_hi. 
    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat_lo : numpy array
        Lower prediction interval.
    y_hat_hi : numpy array
        Higher prediction interval.

    Returns
    -------
    numpy array or double
        Coverage of y_hat

    References
    ----------
    [1] https://www.jstor.org/stable/2629907            
    """ 
    return 100 * np.logical_and(y>=y_hat_lo, y<=y_hat_hi).mean()

In [ ]:
show_doc(coverage)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L465){target="_blank" style="float:right; font-size:smaller"}

### coverage

>      coverage (y:numpy.ndarray, y_hat_lo:numpy.ndarray,
>                y_hat_hi:numpy.ndarray)

Coverage of y with y_hat_lo and y_hat_hi.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| y | ndarray | Observed values. |
| y_hat_lo | ndarray | Lower prediction interval. |
| y_hat_hi | ndarray | Higher prediction interval. |
| **Returns** | **Union** | **Coverage of y_hat** |

## Calibration

In [ ]:
#| export
def calibration(
    y: np.ndarray,
    y_hat_hi: np.ndarray, 
) -> Union[float, np.ndarray]:
    """
    Fraction of y that is lower than y_hat_hi. 
    
    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat_hi : numpy array
        Higher prediction interval.

    Returns
    -------
    numpy array or double
        Calibration of y_hat
        
    References
    ----------
    [1] https://www.jstor.org/stable/2629907            
    """ 
    return (y<=y_hat_hi).mean()

In [ ]:
show_doc(calibration)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L494){target="_blank" style="float:right; font-size:smaller"}

### calibration

>      calibration (y:numpy.ndarray, y_hat_hi:numpy.ndarray)

Fraction of y that is lower than y_hat_hi.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| y | ndarray | Observed values. |
| y_hat_hi | ndarray | Higher prediction interval. |
| **Returns** | **Union** | **Calibration of y_hat** |

## CRPS
> $$ \mathrm{sCRPS}(\hat{F}_{\tau}, \mathbf{y}_{\tau}) = \frac{2}{N} \sum_{i}
    \int^{1}_{0}
    \frac{\mathrm{QL}(\hat{F}_{i,\tau}, y_{i,\tau})_{q}}{\sum_{i} | y_{i,\tau} |} dq $$
>
> Where $\hat{F}_{\tau}$ is the an estimated multivariate distribution, and $y_{i,\tau}$
    are its realizations. 

In [ ]:
#| export
def scaled_crps(
    y: np.ndarray,
    y_hat: np.ndarray, 
    quantiles: np.ndarray, 
    weights: Optional[np.ndarray] = None,
    axis: Optional[int] = None
) -> Union[float, np.ndarray]:
    """Scaled Continues Ranked Probability Score
    
    Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
    to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
    This metric averages percentual weighted absolute deviations as 
    defined by the quantile losses.


    Parameters
    ----------
    y : numpy array
        Observed values.
    y_hat : numpy array
        Predicted values.    
    quantiles : numpy array
        Quantiles to compare against.
    weights : numpy array, optional (default=None)
        Weights for weighted average.
    axis : int, optional (default=None)
        Axis or axes along which to average a. 
        The default, axis=None, will average over all of the elements of 
        the input array. If axis is negative it counts from the last to first.

    Returns
    -------
    numpy array or double.
        Scaled crps along the specified axis.

    References
    ----------
    [1] https://proceedings.mlr.press/v139/rangapuram21a.html      
    """ 
    eps = np.finfo(float).eps
    norm  = np.sum(np.abs(y))
    loss  = mqloss(y=y, y_hat=y_hat, quantiles=quantiles, weights=weights, axis=axis)
    loss  = 2 * loss * np.sum(np.ones(y.shape)) / (norm + eps)
    return loss

In [ ]:
show_doc(scaled_crps)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L520){target="_blank" style="float:right; font-size:smaller"}

### scaled_crps

>      scaled_crps (y:numpy.ndarray, y_hat:numpy.ndarray,
>                   quantiles:numpy.ndarray,
>                   weights:Optional[numpy.ndarray]=None,
>                   axis:Optional[int]=None)

Scaled Continues Ranked Probability Score

Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
This metric averages percentual weighted absolute deviations as 
defined by the quantile losses.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| y | ndarray |  | Observed values. |
| y_hat | ndarray |  | Predicted values.     |
| quantiles | ndarray |  | Quantiles to compare against. |
| weights | Optional | None | Weights for weighted average. |
| axis | Optional | None | Axis or axes along which to average a. <br>The default, axis=None, will average over all of the elements of <br>the input array. If axis is negative it counts from the last to first. |
| **Returns** | **Union** |  | **Scaled crps along the specified axis.** |